In [92]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/u/shuhan/projects/vla')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
from src.models.vlas.cont_obs_token_action import ContObsTokenActionVLA
from transformers import AutoModelForCausalLM

llm_backbone = AutoModelForCausalLM.from_pretrained('gpt2')

hidden_dim = 768
obs_dim = 25
num_actions = 5
mlp_layers = 2

model = ContObsTokenActionVLA(llm_backbone, obs_dim, num_actions, hidden_dim, mlp_layers)

In [54]:
import torch
import os

ckpt = os.path.expanduser('~/results/vla/quick_run/action_only/lightning_logs/version_1/checkpoints/epoch=57-step=18154.ckpt')

lg_ckpt = torch.load(ckpt, map_location='cpu', weights_only=True)['state_dict']
ori_ckpt = {}
for k, v in lg_ckpt.items():
    if k.startswith('vla.'):
        ori_ckpt[k[4:]] = v

model.load_state_dict(ori_ckpt)

<All keys matched successfully>

In [89]:
from IPython.display import HTML
import tqdm
import numpy as np
import gymnasium
import highway_env
from matplotlib import pyplot as plt
%matplotlib inline


observations = []
actions = []
reward_names = ['collision_reward', 'right_lane_reward', 'high_speed_reward', 'on_road_reward']
rewards = {name: [] for name in reward_names}

env = gymnasium.make('highway-fast-v0', render_mode='rgb_array')
obs, _ = env.reset()

observations.append(obs)

rollout_length = 100  # Adjust

for _ in range(rollout_length):
    obs_input = torch.tensor(np.stack(observations), dtype=torch.float32)
    past_actions = torch.tensor(actions)

    action = model.predict_action(obs_input, past_actions).argmax().item()

    obs, reward, done, truncated, info = env.step(action)

    observations.append(obs)
    actions.append(action)

    for name in reward_names:
        rewards[name].append(info['rewards'][name])

    if done or truncated:
        if truncated:
            print('rollout successfully finished!')
        else:
            print('rollout failed!')
        break

env.close()

avg_rewards = {name: np.mean(rewards[name]) for name in reward_names}
rollout_steps = len(observations) - 1


rollout successfully finished!


In [90]:
avg_rewards

{'collision_reward': 0.0,
 'right_lane_reward': 0.7,
 'high_speed_reward': 0.7020791344421672,
 'on_road_reward': 1.0}

In [91]:
rollout_steps

30